In [1]:
import numpy as np
import pandas as pd
import json
import re
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
def show_nans(df, limit):
    missing = df.isnull().mean()*100
    return missing.loc[lambda x : x > limit] 

In [5]:
def n_sil(x):
    for i in range(0, len(x)):
        if isinstance(x[i], list):
            for a in x[i]:
                if a == "\n":
                    x[i].remove(a)
                elif "\n" in a:
                    x[i].remove(a)
                    b = a.replace("\n", "")
                    x[i].append(b)
        elif isinstance(x[i], str):
            x[i] = x[i].replace("\n", "")

In [6]:
def Nan_to_list(data, column_name):
    NaN_rows = data[column_name].isnull()
    # Change the type of NaNs from 'float' to (empty) 'list'
    data.loc[NaN_rows, column_name] = data.loc[NaN_rows, column_name].apply(lambda x: [])

In [7]:
def unique_values(data, column_name):
    unique_vals = set()
    for rows in data[column_name]:
        unique_vals.update(rows)
    return list(unique_vals)

In [8]:
def fill_most(oto, group_col, col_name):
    for group in list(oto[group_col].unique()):
        try:
            grp_inx = list(oto[oto[group_col] == group][col_name].index)
            oto[col_name].iloc[grp_inx] = oto[col_name].iloc[grp_inx].fillna(oto[oto[group_col]==group][col_name].mode()[0])
        except Exception as e:
            print("{} 'da {} Error Oldu".format(group, e))
            print("devam ediliyor")

In [9]:
veri = []
for line in open("scout_car.json", "r"):
    veri.append(json.loads(line))
    
#pd.read_json("scout_car.json", lines = True)

In [10]:
df = pd.DataFrame(veri)

In [11]:
df.shape

(15919, 54)

In [12]:
df.rename(columns={"\nComfort & Convenience\n":"Comfort & Convenience", "\nEntertainment & Media\n":"Entertainment & Media", "\nExtras\n":"Extras", "\nSafety & Security\n":"Safety & Security"}, inplace=True)

In [13]:
#Verilerin %60ından fazlası null olan sütunlar düşürüldü.
df.drop(["null", 
         "kW", 
         "Electricity consumption", 
         "Last Service Date", 
         "Other Fuel Types", 
         "Availability", 
         "Last Timing Belt Service Date", 
         "Available from", 
         "Inspection new", 
         "Emission Label", 
         "Model Code"], axis=1, inplace=True)

#Tekrarlı veya gereksiz sütunlar düşürüldü.Body=body_type idi.
df.drop(["Body", "Offer Number", "prev_owner", "Non-smoking Vehicle", "Weight", "url", "Full Service", "First Registration"], axis=1, inplace=True)

In [14]:
df.shape

(15919, 35)

In [15]:
for i in df.columns:
    Nan_to_list(df, i)

In [16]:
yeni_list = []
for i in df.columns[9:]:
    yeni_list.append(i)

In [17]:
yeni_list

['Previous Owners',
 'Next Inspection',
 'Warranty',
 'Make',
 'Model',
 'Body Color',
 'Paint Type',
 'Body Color Original',
 'Upholstery',
 'Nr. of Doors',
 'Nr. of Seats',
 'Gearing Type',
 'Displacement',
 'Cylinders',
 'Drive chain',
 'Fuel',
 'Consumption',
 'CO2 Emission',
 'Emission Class',
 'Comfort & Convenience',
 'Entertainment & Media',
 'Extras',
 'Safety & Security',
 'description',
 'Gears',
 'Country version']

In [18]:
for i in yeni_list:
    n_sil(df[i])

In [19]:
for i in yeni_list:
    n_sil(df[i])

In [20]:
df_ilk = df.copy()

# Previous Owner

In [21]:
def prev_own(df, clm):   #Prev Owner içinde sadece "0", "1", "2", "3", "4"(prev_owner'da sadece bu değerler olduğu için) olanları alır. 
    for i in range(0, len(df[clm])):
        if type(df[clm][i]) == list:
            for a in df[clm][i]:
                if a in ["0", "1", "2", "3", "4"]:
                    df[clm][i] = a

In [22]:
prev_own(df, "Previous Owners")

In [35]:
df["Previous Owners"].value_counts(dropna=False)

1      8294
NaN    6640
2       778
0       188
3        17
4         2
Name: Previous Owners, dtype: int64

In [ ]:
df["Previous Owners"][df.Type=="New"] = df["Previous Owners"][df.Type=="New"].fillna("0")

# Warranty

In [24]:
def warranty(df, clm):   #Warranty sütunundaki alakasız bilgileri siler.
    for i in range(0, len(df[clm])):
        if type(df[clm][i]) == list:
            df[clm][i] = [x for x in df[clm][i] if x.isnumeric() or "month" in x]
        elif type(df[clm][i]) == str:
            b = []
            b.append(df[clm][i])
            df[clm][i] = [x for x in b if x.isnumeric() or "month" in x]

In [25]:
warranty(df, "Warranty")

# Liste Boş Elemanları Silme

In [26]:
def boslistsil(df, clm):       # Sütundaki listelerin içindeki boşlukları siler.
    for i in range(0, len(df[clm])):
        if type(df[clm][i]) == list:
            df[clm][i] = [x for x in df[clm][i] if len(x)!=0]

In [27]:
for i in ["CO2 Emission",
         "Emission Class",
         "Type"]:
    boslistsil(df, i)

# Listeler Stringe

In [28]:
def liststring(df, col):             #Listeleri stringlere dönüştürür.
    df[col] = [','.join(l) if type(l) == list else l for l in df[col]]
    
def emptytonan(df, col):             #Stringlerdeki boşlukları NaN'a dönüştürür.
    df[col] = [np.NaN if len(l)==0 else l for l in df[col]]

In [34]:
for i in ["Previous Owners",
          "Warranty", 
          "Model", 
          "Body Color", 
          "Body Color Original", 
          "Upholstery", 
          "Nr. of Doors",
          "Nr. of Seats",
          "Gearing Type",
          "Displacement",
          "Cylinders",
          "Drive chain",
          "CO2 Emission",
          "Emission Class",
          "Paint Type", 
          "Country version", 
          "Gears"]:
    try:
        liststring(df, i)
        emptytonan(df, i)
    except:
        print("{} 'da {} hatası çıktı".format(i, Exception))

Previous Owners 'da <class 'Exception'> hatası çıktı
Warranty 'da <class 'Exception'> hatası çıktı


# Fuel

In [42]:
liststring(df, "Fuel")

In [43]:
df.Fuel = df.Fuel.str.split("/")

In [44]:
df.Fuel.value_counts()

[Diesel (Particulate Filter)]                                                                                         4315
[Super 95]                                                                                                            3338
[Gasoline]                                                                                                            3175
[Diesel]                                                                                                              2984
[Super 95 ,  Regular, Benzine 91]                                                                                      424
                                                                                                                      ... 
[Super Plus 98 ,  Super E10 95]                                                                                          1
[Regular, Benzine 91 ,  Super 95 ,  Regular, Benzine E10 91 ,  Super E10 95 ,  Super Plus 98 ,  Super Plus E10 98]       1
[Super 95 ,  Reg

In [45]:
for i in range(0, len(df.Fuel)):
    df.Fuel[i] = [b.strip() for b in df.Fuel[i]]

In [46]:
liststring(df, "Fuel")

# Type

In [47]:
liststring(df, "Type")

In [48]:
df.Type = df.Type.str.split("/")

In [49]:
for i in range(0, len(df.Type)):
    df.Type[i] = [b.strip() for b in df.Type[i]]

In [50]:
liststring(df, "Type")

In [51]:
def conversion(df, clm, wtc=[]):
    for i in wtc:
        df[clm][df[clm].str.contains(i, regex=False)] = i

In [52]:
conversion(df, "Type", wtc=["Used", "Pre-registered", "New", "Employee's car", "Demonstration"])

In [86]:
df.Type[df.Type == ""] = "Used"

# Warranty

In [53]:
fill_most(df, "make_model", "Warranty") #Warranty NANları silindi.

Audi A2 'da index out of bounds Error Oldu
devam ediliyor


In [54]:
df.Warranty[df.Warranty.isnull()] = "12 months"

# body_type

In [55]:
emptytonan(df, "body_type")

In [56]:
fill_most(df, "make_model", "body_type")

# vat

In [57]:
liststring(df, "vat")

In [58]:
emptytonan(df, "vat")

In [59]:
fill_most(df, "make_model", "vat")

# km

In [60]:
df.km = df.km.str.strip(" km")

In [61]:
df.km = df.km.str.replace(",", "")

In [62]:
df.km[df.km.str[0]== "-"] = np.NaN

In [63]:
fill_most(df, "make_model", "km")

# registration

In [64]:
df.registration[df.registration.str[0]== "-"] = np.NaN

In [65]:
fill_most(df, "km", "registration")

0 'da index out of bounds Error Oldu
devam ediliyor
281 'da index out of bounds Error Oldu
devam ediliyor
196 'da index out of bounds Error Oldu
devam ediliyor
141 'da index out of bounds Error Oldu
devam ediliyor
68485 'da index out of bounds Error Oldu
devam ediliyor
81800 'da index out of bounds Error Oldu
devam ediliyor
142 'da index out of bounds Error Oldu
devam ediliyor
39962 'da index out of bounds Error Oldu
devam ediliyor
32084 'da index out of bounds Error Oldu
devam ediliyor
20768 'da index out of bounds Error Oldu
devam ediliyor
34164 'da index out of bounds Error Oldu
devam ediliyor
325 'da index out of bounds Error Oldu
devam ediliyor
89982 'da index out of bounds Error Oldu
devam ediliyor
77 'da index out of bounds Error Oldu
devam ediliyor
89692 'da index out of bounds Error Oldu
devam ediliyor
784 'da index out of bounds Error Oldu
devam ediliyor
497 'da index out of bounds Error Oldu
devam ediliyor


In [66]:
df.registration.iloc[df[df.km=="0"].index] = df.registration.iloc[df[df.km=="0"].index].fillna("01/2020")

In [67]:
fill_most(df, "make_model", "registration")

# hp

In [73]:
dfden = df.copy()

In [80]:
df.hp = df.hp.str.replace(" kW", "")

In [81]:
df.hp[df.hp.str[0]== "-"] = np.NaN

In [82]:
fill_most(df, "make_model", "hp")

In [109]:
df["Previous Owners"][df.Type=="New"] = df["Previous Owners"][df.Type=="New"].fillna("0")

In [114]:
dfden = df.copy()

In [115]:
fill_most(dfden, "registration", "Previous Owners")

11/2019 'da index out of bounds Error Oldu
devam ediliyor
12/2019 'da index out of bounds Error Oldu
devam ediliyor


In [117]:
dfden["Previous Owners"] = dfden["Previous Owners"].fillna("1")

In [118]:
dfden["Previous Owners"].value_counts(dropna=False)

1    13228
0     1894
2      778
3       17
4        2
Name: Previous Owners, dtype: int64

In [111]:
df["Previous Owners"].value_counts(dropna=False)

1      8294
NaN    5169
0      1659
2       778
3        17
4         2
Name: Previous Owners, dtype: int64

In [69]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [88]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,15869,15870,15871,15872,15873,15874,15875,15876,15877,15878,15879,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889,15890,15891,15892,15893,15894,15895,15896,15897,15898,15899,15900,15901,15902,15903,15904,15905,15906,15907,15908,15909,15910,15911,15912,15913,15914,15915,15916,15917,15918
make_model,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,...,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace
short_description,Sportback 1.4 TDI S-tronic Xenon Navi Klima,1.8 TFSI sport,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,1.4 TDi Design S tronic,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,1.6 TDI Sport DSG *SHZ*Xenon*Navi*Bose*PDC*,Sportback 1.6 TDI S-TRONIC S-LINE*NAVI|XENON*,Sportback 1.4 TDI admired S tronic,SPB 1.6 TDI S-tronic Metal plus + Xeno,SPORTBACK TFSI ULTRA 95 S-TRONIC AMB.,SPORTBACK1.6 TDI 116 CV S tronic Metal plus,Sportback Sport »1.4 TFSI|Xenon|Navi|EPH+|SHZ|...,Sportback 1.4 TFSI S-tronic *S line*NAV*XEN*,1.4 TFSI 150ch COD Ambition Luxe S tronic 7,Sportback 1.0 TFSI S-TRonic*Navi*Xenon*APS* Sport,SPB 1.6 TDI 116 CV S tronic Sport,SPB 1.6 TDI 116 CV Design S-Line SPB 1.6 TDI 1...,1.4 TDi S tronic*S-Line*Navigatie*Euro 6b*,1.0 TFSI 95pk Automaat Adrenalin,Sportback 1.6 TDI 116 CV S tronic Metal plus,SPB 1.6 TDI 116 CV S tronic Admired S-Line,Sportback Sport 1.4 TFSI S tronic XENON NAVI,1.4 TFSI S tronic,SPB 1.6 TDI 116 CV S tronic Metal plus,SPB 1.0 TFSI ultra S tronic!! 3.000KM! COME NU...,1.0 TFSI ultra Sportback TÜV NEU,1.0 TFSI *PDC*SHZ*Klimaautomatik*,Sportback 1.0 TFSI Attraction,/S1 SPB 1.4 TDI S tronic Admired,Sportback basis,Sportback 1.4TDI Attraction,1.4 TDI ultra Klimaautomatik Xenon,1.0 TFSI ultra KLIMA LM ZV,Sportback 1.4TDI Attraction,SPB 1.4 TDI 90cv EURO6 NEOPATENTATI,1.0 TFSI ultra KLIMA FSE SHZ EU6,1.0 TFSI ultra *GRA*Klima*,Sportback 1.4TDI Attraction,1.0 TFSI ultra sport,"(€8950,-excl.VAT/Guarantee) SB 1.4 TDi Ultra Eur6",3-Türer 1.0 TFSI 5-Gang GRA Sportlenkrad EURO 6,1.4 TDI ultra Sportback *1.Hand*Navi*Pdc*,1.4 TDI ultra Sportback *1.Hand*Navi*Pdc*,"(€8900,-excl.VAT/Guarantee) SB 1.4TDI Ultra Eu...",Sportback 1.0 TDI ultra,"1.0 TFSI Ultra*EURO6*Klimaautomatik*LM16""*",1.4 TDi,Sportback 1.6TDI Attraction,SPORT KLIMA+SHZ+PDC+16-ZOLL-ALU+EURO 6,1.0 TFSI basis,...,2.0 dCi 200 Intens 5p.,INITIALE PARIS TCe 225 EDC GPF,V Initiale Paris DCI 200 EDC Cruising-Pak,INITIALE PARIS TCE 225 EDC PF EU6DT,225pk Intens / Nieuw / PACK CRUISING,V Limited Vollausstattung,INITIALE Paris BLUE dCi 200 EDC ABS NAVI,1.8 TCe Intens 5p.,Initiale Paris ENERGY TCe 200 EDC LED,Initiale Paris 2.0 BLUE dCi 200 EDC 7-Sitzer L...,INITIALE PARIS TCe 225 EDC GPF,BLUE dCi 200 EDC INITIALE PARIS,INITIALE PARIS TCe 225 EDC GPF,V 1.8 TCe 225 EDC Initiale Paris,RENAULT INITIALE BLUE DCI 200PS,TCe 225 EDC GPF INITIALE PARIS,Initiale Par

# NOTLAR

In [ ]:
def boslistsil(df, clm):
    for i in range(0, len(df[clm])):
        if type(df[clm][i]) == list:
            df[clm][i] = [x for x in df[clm][i] if len(x)!=0]
        else:
            df[clm][i] = df[clm][i]

In [ ]:
# TAMAM OLANLAR
    -make_model
    -body_type
    -Gearing Type
    -Previous Owners
    -Warranty
    -Make
    -Model
    -CO2 Emission
    -Emission Class
    -Cylinders

In [ ]:
.str.method,
.contains(),
.extract(),
.to_datetime(),
.get_dummies(),
.add_prefix(),
.sample(),
regex,
.to_numeric(),
.isin(),
.corr().

In [92]:
s = pd.Series(list('abca'))
s

0    a
1    b
2    c
3    a
dtype: object

In [91]:
pd.get_dummies(s)

,a,b,c
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0


In [177]:
ee = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5]})
ee

,year,month,day
0,2015,2,4
1,2016,3,5


In [178]:
pd.to_datetime(ee)

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]

In [105]:
s3 = pd.Series(['a1', 'b2', 'c3'])
s3

0    a1
1    b2
2    c3
dtype: object

In [106]:
s3.str.extract(r'([ab])(\d)')

,0,1
0,a,1
1,b,2
2,NaN,NaN


In [107]:
s1 = pd.Series(['Mouse', 'dog', 'house and parrot', '23', np.NaN])
s1

0               Mouse
1                 dog
2    house and parrot
3                  23
4                 NaN
dtype: object

In [108]:
s1.str.contains('og', regex=False)

0    False
1     True
2    False
3    False
4      NaN
dtype: object

In [113]:
s2 = pd.Series(["A_Str_Series"])
s2

0    A_Str_Series
dtype: object

In [115]:
s2.str.split("_")

0    [A, Str, Series]
dtype: object

# Stringe Çevirme Bölümü

In [199]:
a = ["ay", "günes", "misir"]

In [200]:
", ".join(a)

'ay, günes, misir'

In [201]:
def list_string(liste, seperator=' '):
    return seperator.join(liste)

In [203]:
list_string(a, ", ")

'ay, günes, misir'

In [204]:
df['liststring'] = [','.join(map(str, l)) for l in df['Country version']]

In [ ]:
def liststring(dfcol):
    dfcol = [', '.join(l) for l in dfcol]

In [211]:
df['liststring2'] = [','.join(l) for l in df['Country version']]

In [98]:
def try_join(l):
    try:
        return ','.join(map(str, l))
    except TypeError:
        return np.nan

In [99]:
df['liststring'] = [try_join(l) for l in df['Body Color']]

In [187]:
map(str, ["ay", "günes", "misir"])

In [189]:
for i in map(str, ["ay", "günes", "misir"]):
    print(type(i))

<class 'str'>
<class 'str'>
<class 'str'>
